In [1]:
%matplotlib inline

## main packages
import dask.array as da
from dask_glm import gradient
from multipledispatch import dispatch
import matplotlib.pyplot as plt
import numpy as np

## utility functions
from utilities import dot, absolute, sigmoid

In [2]:
## size of problem (no. observations)
N = 40
chunks = 2
seed = 20009

## Create some data (tall and very skinny)

In [3]:
def data_create(seed, N, chunks):
    da.random.seed(seed)
    X = da.random.random((N,2), chunks=chunks)
    y = (X.dot(np.array([[1.5,-3]]).T)).map_blocks(sigmoid) + .001*da.random.normal(size=(N,1), chunks=chunks)
    return X,y

X,y = data_create(seed, N, chunks)

In [4]:
# np.random.seed(20009)
# X = np.random.random((N,2))
# y = np.array([sigmoid(x) + 0.001*np.random.normal(0,.1) for x in X.dot(np.array([[1.5,-3]]).T)])
# X = da.from_array(X, chunks=2)
# y = da.from_array(y, chunks=2)

Should profile the Hessian computation, it's possible there's a more efficient way to construct without going through the diagonal matrix construction.

## Dask Inspect some Newton constructs

In [5]:
beta = np.ones((2,1))

## 
p = (X.dot(beta)).map_blocks(sigmoid)
W = da.diag((p*(1-p))[:,0])
hessian = dot(X.T.dot(W),X)
grad = X.T.dot(y-p)
#hessian.visualize()

In [6]:
hessian.compute()

array([[ 2.30984176,  1.83997696],
       [ 1.83997696,  2.515287  ]])

In [7]:
grad.compute()

array([[ -8.31844271],
       [-11.47213857]])

Is there a benefit / difference between 

- Running `da.compute()` on Hessian and gradient then calling `np.linalg.lstsq()` 
- Calling `da.linalg.lstsq()` on two dask arrays

In [8]:
def newton_step(curr, X, y, lam=None):
    '''One naive step of Newton's Method'''
    
    ## compute necessary objects
    p = (X.dot(curr)).map_blocks(sigmoid)
    W = da.diag((p*(1-p))[:,0])
    hessian = dot(X.T.dot(W),X) # shape (parameters x parameters)
    grad = X.T.dot(y-p)
    
    ## regularization
    if lam:
        step, *_ = da.linalg.lstsq(hessian + lam*np.eye(curr.shape[0]), grad)
    else:
        step, *_ = da.linalg.lstsq(hessian, grad)
        
    ## update
    beta = curr + step
    
    return beta.compute()

def check_coefs_convergence(beta_old, beta_new, tol, iters, max_iter=50):
    '''Checks whether the coefficients have converged in the l-infinity norm.
    Returns True if they have converged, False otherwise.'''
    
    coef_change = absolute(beta_old - beta_new)
    
    return not (np.any(coef_change>tol) & (iters < max_iter))

In [9]:
beta2 = newton_step(beta, X, y)
beta2

array([[ 1.07639001],
       [-3.61684669]])

The sigmoid function can create numerical overflow sometimes (switch `.001` to `0` in the data generation step to see it, with `N=4` and `chunks=2`).

In [10]:
def newton(X,y,beta):
    
    iter_count,lam = 0,0
    coefs_converged = False

    while not coefs_converged:

        beta_old = beta
        beta = newton_step(beta, X, y, lam=lam)
        iter_count += 1

        coefs_converged = check_coefs_convergence(beta_old, beta, 1e-4, iter_count)

    print('Iterations : {}'.format(iter_count))
    print('Beta : {}'.format(beta[:,0]))

## Run some Experiments

In [11]:
%%time
newton(X,y,beta)

Iterations : 6
Beta : [ 1.49919986 -2.99742612]
CPU times: user 2.89 s, sys: 339 ms, total: 3.23 s
Wall time: 3.06 s


In [12]:
%%time
gradient.gradient(X,y[:,0])

##       -f        |df/f|    |dx/x|    step
----------------------------------------------
 1  2.495582e+01  9.99e-02  5.00e-01  1.0e-01
 2  2.422214e+01  2.94e-02  2.65e-01  1.2e-01
 3  2.381012e+01  1.70e-02  1.86e-01  1.6e-01
 4  2.343089e+01  1.59e-02  1.71e-01  2.0e-01
 5  2.307541e+01  1.52e-02  1.61e-01  2.4e-01
 6  2.276403e+01  1.35e-02  1.48e-01  3.1e-01
 7  2.251322e+01  1.10e-02  1.32e-01  3.8e-01
 8  2.233067e+01  8.11e-03  1.15e-01  4.8e-01
 9  2.221328e+01  5.26e-03  9.61e-02  6.0e-01
10  2.214872e+01  2.91e-03  7.64e-02  7.5e-01
11  2.211984e+01  1.30e-03  5.64e-02  9.3e-01
12  2.211023e+01  4.35e-04  3.71e-02  1.2e+00
13  2.210923e+01  4.50e-05  1.18e-02  7.3e-01
14  2.210890e+01  1.51e-05  9.32e-03  4.5e-01
15  2.210832e+01  2.62e-05  5.88e-03  2.8e-01
16  2.210815e+01  7.44e-06  2.95e-03  3.6e-01
17  2.210803e+01  5.41e-06  2.73e-03  4.4e-01
18  2.210796e+01  3.35e-06  2.92e-03  5.6e-01
19  2.210791e+01  2.41e-06  2.01e-03  3.5e-01
20  2.210788e+01  1.30e-06  1.63e-0

array([ 1.49919915, -2.99742543])

In [13]:
X,y = data_create(seed, 10000, chunks=100)

In [14]:
%%time
newton(X,y,beta)

Iterations : 5
Beta : [ 1.50014406 -3.00026053]
CPU times: user 1min 6s, sys: 8.97 s, total: 1min 15s
Wall time: 1min 11s


In [16]:
%%time
gradient.gradient(X,y[:,0])

##       -f        |df/f|    |dx/x|    step
----------------------------------------------
 1  6.198546e+03  1.06e-01  5.00e-01  1.0e-03
 2  5.950491e+03  4.00e-02  3.49e-01  1.3e-03
 3  5.803968e+03  2.46e-02  2.33e-01  1.6e-03
 4  5.716700e+03  1.50e-02  1.98e-01  2.0e-03
 5  5.692881e+03  4.17e-03  1.78e-01  2.4e-03
 6  5.652310e+03  7.13e-03  1.50e-01  1.5e-03
 7  5.630210e+03  3.91e-03  6.54e-02  9.5e-04
 8  5.625036e+03  9.19e-04  2.75e-02  1.2e-03
 9  5.620660e+03  7.78e-04  2.76e-02  1.5e-03
10  5.617261e+03  6.05e-04  2.68e-02  1.9e-03
11  5.614932e+03  4.15e-04  2.47e-02  2.3e-03
12  5.613592e+03  2.39e-04  2.13e-02  2.9e-03
13  5.613108e+03  8.62e-05  1.78e-02  3.6e-03
14  5.612875e+03  4.15e-05  7.01e-03  1.1e-03
15  5.612808e+03  1.20e-05  3.33e-03  1.4e-03
16  5.612756e+03  9.26e-06  3.18e-03  1.8e-03
17  5.612719e+03  6.61e-06  3.11e-03  2.2e-03
18  5.612699e+03  3.43e-06  3.10e-03  2.8e-03
19  5.612690e+03  1.61e-06  2.17e-03  1.7e-03
20  5.612687e+03  4.89e-07  2.17e-0

array([ 1.50014338, -3.00025956])

In [24]:
X, y = X.rechunk(chunks=50), y.rechunk(chunks=50)

In [25]:
%%time
newton(X,y,beta)

Iterations : 5
Beta : [ 1.50014406 -3.00026053]
CPU times: user 4min 52s, sys: 30.3 s, total: 5min 22s
Wall time: 5min 11s


In [26]:
%%time
gradient.gradient(X,y[:,0])

##       -f        |df/f|    |dx/x|    step
----------------------------------------------
 1  6.198546e+03  1.06e-01  5.00e-01  1.0e-03
 2  5.950491e+03  4.00e-02  3.49e-01  1.3e-03
 3  5.803968e+03  2.46e-02  2.33e-01  1.6e-03
 4  5.716700e+03  1.50e-02  1.98e-01  2.0e-03
 5  5.692881e+03  4.17e-03  1.78e-01  2.4e-03
 6  5.652310e+03  7.13e-03  1.50e-01  1.5e-03
 7  5.630210e+03  3.91e-03  6.54e-02  9.5e-04
 8  5.625036e+03  9.19e-04  2.75e-02  1.2e-03
 9  5.620660e+03  7.78e-04  2.76e-02  1.5e-03
10  5.617261e+03  6.05e-04  2.68e-02  1.9e-03
11  5.614932e+03  4.15e-04  2.47e-02  2.3e-03
12  5.613592e+03  2.39e-04  2.13e-02  2.9e-03
13  5.613108e+03  8.62e-05  1.78e-02  3.6e-03
14  5.612875e+03  4.15e-05  7.01e-03  1.1e-03
15  5.612808e+03  1.20e-05  3.33e-03  1.4e-03
16  5.612756e+03  9.26e-06  3.18e-03  1.8e-03
17  5.612719e+03  6.61e-06  3.11e-03  2.2e-03
18  5.612699e+03  3.43e-06  3.10e-03  2.8e-03
19  5.612690e+03  1.61e-06  2.17e-03  1.7e-03
20  5.612687e+03  4.89e-07  2.17e-0

array([ 1.50014338, -3.00025956])

In [27]:
X,y = data_create(seed, 100000, chunks=1000)

In [28]:
%%time
newton(X,y,beta)

Iterations : 5
Beta : [ 1.50007803 -3.00003909]
CPU times: user 11min 3s, sys: 4min 29s, total: 15min 33s
Wall time: 6min 5s


In [29]:
%%time
gradient.gradient(X,y[:,0])

##       -f        |df/f|    |dx/x|    step
----------------------------------------------
 1  6.209785e+04  1.04e-01  5.00e-01  1.0e-04
 2  5.959951e+04  4.02e-02  3.54e-01  1.3e-04
 3  5.813401e+04  2.46e-02  2.36e-01  1.6e-04
 4  5.727424e+04  1.48e-02  2.02e-01  2.0e-04
 5  5.711269e+04  2.82e-03  1.87e-01  2.4e-04
 6  5.664658e+04  8.16e-03  1.64e-01  1.5e-04
 7  5.637851e+04  4.73e-03  7.28e-02  9.5e-05
 8  5.632638e+04  9.25e-04  2.79e-02  1.2e-04
 9  5.628271e+04  7.75e-04  2.76e-02  1.5e-04
10  5.624879e+04  6.03e-04  2.68e-02  1.9e-04
11  5.622555e+04  4.13e-04  2.47e-02  2.3e-04
12  5.621235e+04  2.35e-04  2.15e-02  2.9e-04
13  5.620952e+04  5.03e-05  1.97e-02  3.6e-04
14  5.620496e+04  8.11e-05  1.03e-02  1.1e-04
15  5.620422e+04  1.33e-05  3.86e-03  1.4e-04
16  5.620369e+04  9.41e-06  3.42e-03  1.8e-04
17  5.620333e+04  6.37e-06  3.40e-03  2.2e-04
18  5.620323e+04  1.75e-06  3.85e-03  2.8e-04
19  5.620310e+04  2.34e-06  3.49e-03  1.7e-04
20  5.620294e+04  2.81e-06  1.79e-0

array([ 1.50007742, -3.00003817])

In [33]:
X,y = data_create(seed, 1e6, chunks=1e4)

In [34]:
%%time
gradient.gradient(X,y[:,0])

##       -f        |df/f|    |dx/x|    step
----------------------------------------------
 1  6.204716e+05  1.05e-01  5.00e-01  1.0e-05
 2  5.952461e+05  4.07e-02  3.55e-01  1.3e-05
 3  5.805687e+05  2.47e-02  2.36e-01  1.6e-05
 4  5.720201e+05  1.47e-02  2.03e-01  2.0e-05
 5  5.676665e+05  7.61e-03  1.15e-01  1.2e-05
 6  5.651661e+05  4.40e-03  8.42e-02  1.5e-05
 7  5.635057e+05  2.94e-03  7.42e-02  1.9e-05
 8  5.627526e+05  1.34e-03  7.51e-02  2.4e-05
 9  5.620893e+05  1.18e-03  5.56e-02  1.5e-05
10  5.618287e+05  4.64e-04  4.55e-02  1.9e-05
11  5.615258e+05  5.39e-04  2.70e-02  1.2e-05
12  5.614304e+05  1.70e-04  1.40e-02  1.5e-05
13  5.613613e+05  1.23e-04  1.27e-02  1.8e-05
14  5.613166e+05  7.97e-05  1.26e-02  2.3e-05
15  5.612942e+05  3.99e-05  7.42e-03  1.4e-05
16  5.612802e+05  2.49e-05  6.10e-03  1.8e-05
17  5.612717e+05  1.52e-05  6.16e-03  2.2e-05
18  5.612660e+05  1.02e-05  3.92e-03  1.4e-05
19  5.612629e+05  5.41e-06  2.92e-03  1.7e-05
20  5.612611e+05  3.26e-06  2.93e-0

array([ 1.50000934, -3.00003526])

In [36]:
X,y = data_create(seed, 1e7, chunks=1e4)

In [37]:
%%time
gradient.gradient(X,y[:,0])

##       -f        |df/f|    |dx/x|    step
----------------------------------------------
 1  6.206216e+06  1.05e-01  5.00e-01  1.0e-06
 2  5.954144e+06  4.06e-02  3.56e-01  1.3e-06
 3  5.807726e+06  2.46e-02  2.37e-01  1.6e-06
 4  5.722567e+06  1.47e-02  2.04e-01  2.0e-06
 5  5.678526e+06  7.70e-03  1.16e-01  1.2e-06
 6  5.653389e+06  4.43e-03  8.50e-02  1.5e-06
 7  5.636763e+06  2.94e-03  7.49e-02  1.9e-06
 8  5.629443e+06  1.30e-03  7.64e-02  2.4e-06
 9  5.622563e+06  1.22e-03  5.74e-02  1.5e-06
10  5.619950e+06  4.65e-04  4.71e-02  1.9e-06
11  5.616711e+06  5.76e-04  2.82e-02  1.2e-06
12  5.615735e+06  1.74e-04  1.43e-02  1.5e-06
13  5.615037e+06  1.24e-04  1.29e-02  1.8e-06
14  5.614590e+06  7.96e-05  1.29e-02  2.3e-06
15  5.614354e+06  4.20e-05  7.78e-03  1.4e-06
16  5.614211e+06  2.54e-05  6.31e-03  1.8e-06
17  5.614127e+06  1.51e-05  6.42e-03  2.2e-06
18  5.614065e+06  1.11e-05  4.19e-03  1.4e-06
19  5.614034e+06  5.57e-06  3.07e-03  1.7e-06
20  5.614016e+06  3.22e-06  3.09e-0

array([ 1.49999861, -3.00000063])

## Observations

The first order method is less sensititive to chunks and is clearly superior.  Inverting matrices is hard.  Next step: quasi-Newton, and how is performance impacted when we move to `dask.distributed`?